 # DataCreation

In [11]:
# Install the Faker library (if not already installed)
!pip install faker

# Import necessary libraries
import numpy as np
import pandas as pd
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Set the number of records you want to generate
num_records = 1000  # Adjust this number for more data

# Define the expense categories and budgets
categories = [
    "Salaries", "Marketing & Advertising", "Office Supplies & Equipment", 
    "Entertainment", "Rent & Lease", "Insurance", "Professional Services",
    "Software & Subscriptions", "Taxes & Regulatory Fees", "Maintenance & Repairs"
]

# Predefined budgets for each category (theoretical)
budget_dict = {
    "Salaries": 50000, "Marketing & Advertising": 20000, "Office Supplies & Equipment": 5000,
    "Entertainment": 3000, "Rent & Lease": 15000, "Insurance": 8000,
    "Professional Services": 10000, "Software & Subscriptions": 7000,
    "Taxes & Regulatory Fees": 5000, "Maintenance & Repairs": 4000
}

# Create a list to hold the generated records
data = []

# Generate random expense records
for _ in range(num_records):
    category = random.choice(categories)
    amount = round(random.uniform(100, budget_dict[category] * 2), 2)  # Random amount between 100 and double the budget
    budget = budget_dict[category]
    date = fake.date_between(start_date='-1y', end_date='today')  # Random date in the last year
    data.append({"Expense Category": category, "Amount": amount, "Budget": budget, "Date": date})

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Creating historical spend: average spend per category (this will simulate historical data)
historical_averages = df.groupby('Expense Category')['Amount'].transform('mean')
df['Historical Spend'] = historical_averages

# Add a feature to represent budget percentage
df['Budget %'] = (df['Amount'] / df['Budget']) * 100

# Create random labels for insights and recommendations (this simulates your target variable)
insight_labels = ["Efficient Spending", "Over Budget", "Consider Adjusting", "Excellent"]
recommendation_labels = ["Increase Budget", "Reallocate Funds", "No Action Needed", "Cut Down on Expenses"]

# Randomly assign insights and recommendations to each record
df['Insight'] = np.random.choice(insight_labels, size=len(df))
df['Recommendation'] = np.random.choice(recommendation_labels, size=len(df))

# Display the first few rows of the DataFrame
print(df[['Expense Category', 'Amount', 'Budget', 'Historical Spend', 'Insight', 'Recommendation']].head(20))

# Save the dataset to a CSV file
df.to_csv("corporate_expenses_with_insights.csv", index=False)


               Expense Category    Amount  Budget  Historical Spend  \
0      Software & Subscriptions   8867.58    7000       7152.388557   
1                  Rent & Lease  10635.75   15000      16038.713474   
2         Maintenance & Repairs   6687.55    4000       4165.586129   
3                     Insurance  13028.40    8000       7784.518585   
4   Office Supplies & Equipment   8509.33    5000       5403.607333   
5                      Salaries   2630.77   50000      51145.907957   
6                  Rent & Lease  27817.97   15000      16038.713474   
7                     Insurance   2829.54    8000       7784.518585   
8      Software & Subscriptions   1231.54    7000       7152.388557   
9       Taxes & Regulatory Fees   2771.85    5000       5196.293162   
10      Taxes & Regulatory Fees   5899.66    5000       5196.293162   
11                     Salaries  95603.28   50000      51145.907957   
12                     Salaries  84689.46   50000      51145.907957   
13    

# Part 2: Data Manipulation and Analysis

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Label encode the 'Expense Category' feature since it's categorical
label_encoder = LabelEncoder()
df['Expense Category'] = label_encoder.fit_transform(df['Expense Category'])

# Define features and labels
features = ['Expense Category', 'Amount', 'Budget', 'Historical Spend', 'Budget %']
X = df[features]

# We will train two models: one for 'Insight' and one for 'Recommendation'
y_insight = df['Insight']
y_recommendation = df['Recommendation']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train_insight, y_test_insight = train_test_split(X, y_insight, test_size=0.2, random_state=42)
X_train, X_test, y_train_recommendation, y_test_recommendation = train_test_split(X, y_recommendation, test_size=0.2, random_state=42)


# Part 3: Training 


In [13]:
from sklearn.ensemble import RandomForestClassifier

# Train RandomForest for Insight prediction
model_insight = RandomForestClassifier(n_estimators=100, random_state=42)
model_insight.fit(X_train, y_train_insight)

# Train RandomForest for Recommendation prediction
model_recommendation = RandomForestClassifier(n_estimators=100, random_state=42)
model_recommendation.fit(X_train, y_train_recommendation)


RandomForestClassifier(random_state=42)

# evaluation

In [14]:
from sklearn.metrics import classification_report

# Predict insights and recommendations for the test set
insight_predictions = model_insight.predict(X_test)
recommendation_predictions = model_recommendation.predict(X_test)

# Print classification report for Insight predictions
print("Insight Classification Report:")
print(classification_report(y_test_insight, insight_predictions))

# Print classification report for Recommendation predictions
print("Recommendation Classification Report:")
print(classification_report(y_test_recommendation, recommendation_predictions))


Insight Classification Report:
                    precision    recall  f1-score   support

Consider Adjusting       0.17      0.11      0.13        63
Efficient Spending       0.26      0.34      0.29        41
         Excellent       0.15      0.17      0.16        42
       Over Budget       0.24      0.26      0.25        54

          accuracy                           0.21       200
         macro avg       0.21      0.22      0.21       200
      weighted avg       0.20      0.21      0.20       200

Recommendation Classification Report:
                      precision    recall  f1-score   support

Cut Down on Expenses       0.23      0.22      0.22        46
     Increase Budget       0.21      0.20      0.20        56
    No Action Needed       0.25      0.25      0.25        53
    Reallocate Funds       0.29      0.33      0.31        45

            accuracy                           0.24       200
           macro avg       0.24      0.25      0.25       200
        weig

# generate

In [15]:
# Predict insights and recommendations for the first 20 test records
predicted_insights = model_insight.predict(X_test[:20])
predicted_recommendations = model_recommendation.predict(X_test[:20])

# Create a DataFrame to display the predictions alongside the actual values
predicted_df = pd.DataFrame({
    'Expense Category': label_encoder.inverse_transform(X_test['Expense Category'][:20]),
    'Amount': X_test['Amount'][:20],
    'Budget': X_test['Budget'][:20],
    'Predicted Insight': predicted_insights,
    'Predicted Recommendation': predicted_recommendations
})

# Display the first 20 rows with predicted insights and recommendations
print(predicted_df)


                Expense Category    Amount  Budget   Predicted Insight  \
521        Maintenance & Repairs   7058.64    4000         Over Budget   
737                    Insurance  14721.94    8000           Excellent   
740        Professional Services   8496.55   10000  Efficient Spending   
660                Entertainment   5127.86    3000  Consider Adjusting   
411      Marketing & Advertising  27343.88   20000  Consider Adjusting   
678                 Rent & Lease   2347.38   15000         Over Budget   
626      Taxes & Regulatory Fees   4848.23    5000         Over Budget   
513        Professional Services   6093.37   10000  Efficient Spending   
859                     Salaries  85881.71   50000         Over Budget   
136      Taxes & Regulatory Fees   3363.99    5000           Excellent   
811     Software & Subscriptions   8107.19    7000  Consider Adjusting   
76         Maintenance & Repairs   4044.38    4000           Excellent   
636  Office Supplies & Equipment   360